# Chains in LangChain
## Outline
- LLMChain
- Sequential Chains
    - SimpleSequentialChain
    - SequentialChain
- Router Chain

In [51]:
import langchain
import langchain_community
import langchain_ollama
import pandas as pd

print("langchain version:", langchain.__version__)
print("langchain_community version:", langchain_community.__version__)
print("langchain_ollama version:", langchain_ollama.__version__)
print("pandas version:", pd.__version__)


langchain version: 0.3.25
langchain_community version: 0.3.24
langchain_ollama version: 0.3.3
pandas version: 2.2.3


In [52]:
import warnings
warnings.filterwarnings('ignore')

In [53]:
import os
import pandas as pd
df = pd.read_csv('data/Data.csv')

In [54]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


# LLMChain

In [7]:
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [8]:
llm = ChatOllama(
    model = "llama3.1",
    temperature = 0
)

In [9]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [10]:
chain = LLMChain(llm=llm, prompt=prompt)

/tmp/ipykernel_58141/1305865249.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [11]:
product = "Queen Size Sheet set"
chain.run(product)

/tmp/ipykernel_58141/544502284.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


'Here are some suggestions for a company name that specializes in Queen-size sheet sets:\n\n1. **Regal Bedding**: This name plays off the idea of royalty, which fits well with the "Queen" size designation.\n2. **Sheets Royale**: Similar to Regal Bedding, this name emphasizes the regal aspect of the product.\n3. **Queen\'s Comfort**: This name conveys a sense of luxury and comfort, which is perfect for a high-quality sheet set.\n4. **Linen Luxe**: "Luxe" implies high-end quality, while "linen" suggests a focus on soft, breathable fabrics.\n5. **Royal Slumber**: This name evokes feelings of relaxation and tranquility, making it perfect for a company that wants to emphasize the comfort of their products.\n6. **Queen Size Co.**: Simple and straightforward, this name clearly communicates what the company specializes in.\n7. **Dreamweaver Bedding**: This name suggests that the company\'s sheet sets will help customers create their own dreamy sleep experiences.\n8. **The Queen\'s Quilt Co.**:

# SimpleSequentialChain
Single Input Single Output

In [16]:
from langchain.chains import SimpleSequentialChain
llm = ChatOllama(model = "llama3.1",
                 temperature=0.9
                )

In [15]:
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

#chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)


In [17]:
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company: {company_name}"
)

#chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [18]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [19]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Here are some suggestions for names that might fit well for a company that makes queen size sheet sets:

1. **Queenly Comfort**: This name suggests that the products will provide comfort and luxury, which is perfect for a company making high-quality bedding.
2. **Regal Bedding Co.**: "Regal" implies a sense of grandeur and sophistication, fitting for a company producing premium bedding products.
3. **Sleep Royale**: This name combines "sleep" (a key aspect of bedding) with "royale," conveying a regal and majestic quality to the brand.
4. **Queen Size Bliss**: This name is straightforward and attention-grabbing, emphasizing the comfort and happiness that customers can expect from the product.
5. **Luxe Sheets by [Company Name]**: Adding "Luxe" to the beginning of the name instantly conveys a sense of high-end quality, which is perfect for a company producing premium bedding products.
6. **Slumber & Majesty**: This name combines two key asp

"Here is a 20-word description for each of the suggested company names:\n\n1. **Queenly Comfort**: Providing luxurious, high-quality queen size sheet sets designed to promote ultimate comfort and relaxation in every home.\n2. **Regal Bedding Co.**: Crafted with precision and care, Regal Bedding Co.'s premium products redefine the art of sleep and serenity.\n3. **Sleep Royale**: Experience regal slumber with Sleep Royale's exquisite queen size sheet sets, expertly woven for ultimate comfort and tranquility.\n4. **Queen Size Bliss**: Indulge in the purest luxury with Queen Size Bliss's premium bedding collection, designed to bring serenity to your bedroom.\n5. **Luxe Sheets by [Company Name]**: Treat yourself to opulent sleep experiences with Luxe Sheets by [Company Name], the epitome of high-end comfort.\n6. **Slumber & Majesty**: Slumber & Majesty's queen size sheet sets embody the perfect blend of restful slumber and majestic luxury.\n7. **Royal Comfort Co.**: Enveloped in royal comfo

# SequentialChain
Multiple Input Multiple Output

In [20]:
from langchain.chains import SequentialChain

In [27]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

In [28]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )

In [29]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [30]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [31]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [32]:
df.Review[5]

"Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"

In [33]:
review = df.Review[5]
overall_chain(review)

/tmp/ipykernel_58141/1992003631.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': 'Here is the translation of the review to English:\n\n"I find the taste mediocre. The foam doesn\'t hold up, that\'s weird. I buy the same ones in stores and the taste is much better...\n\nOld batch or a fake product?"',
 'summary': 'The reviewer found the product to have a mediocre taste and poor foam quality, suggesting it may be an old or counterfeit batch of the store-bought variety which they prefer.',
 'followup_message': 'Réponse :\n\n"Je suis en désaccord avec cette critique. J\'ai eu l\'occasion d\'essayer ce produit et je pense que son gout est correct. Peut-être qu\'il y a eu une erreur de livraison ou que le rédacteur n\'a pas reçu la version bonne ? En tout cas, je suis plus qu\'enclin à recommander ce produit qui reste pour moi l\'un des meilleurs sur le marché."'}

# Router Chain

In [34]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [35]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]


In [39]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [40]:
llm = ChatOllama(temperature=0,
                 model = "llama3.1")

In [41]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [42]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [44]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to wrap the output with ```json (output)```)>>"""

In [45]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [46]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

/tmp/ipykernel_58141/3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [47]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'black body radiation'}
> Finished chain.


'Black-body radiation! A fundamental concept in thermodynamics and statistical mechanics.\n\nIn simple terms, black-body radiation refers to the thermal radiation emitted by an object that absorbs all incident electromagnetic radiation. In other words, it\'s the radiation emitted by an idealized "perfect absorber" of light.\n\nWhen a body is heated, its atoms or molecules gain kinetic energy and start vibrating more rapidly. As they vibrate, they emit photons, which are particles of light. The frequency (or color) of these photons depends on the temperature of the object. At higher temperatures, the object emits radiation with shorter wavelengths (e.g., ultraviolet, X-rays), while at lower temperatures, it emits longer-wavelength radiation (e.g., infrared).\n\nThe key characteristic of black-body radiation is that it\'s a perfect absorber and emitter of electromagnetic radiation. This means that it absorbs all incident radiation without reflecting any of it, and it also emits radiation

In [48]:
chain.run("What is 2+2")



> Entering new MultiPromptChain chain...
math: {'input': 'What is 2 + 2'}
> Finished chain.


"A simple yet fundamental question!\n\nTo break it down, I'll identify the two components of this problem:\n\n1. The first number: 2\n2. The second number: 2\n3. The operation to be performed: addition (+)\n\nNow, let's answer each component separately:\n\n1. The value of the first number is... 2!\n2. The value of the second number is also... 2!\n3. When you add two numbers together, you simply combine their values.\n\nPutting it all together...\n\n2 + 2 = ?\n\nSince both numbers are equal to 2, I can simply combine them:\n\n2 + 2 = 4\n\nTherefore, the answer is: 4!"

In [49]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is the role of DNA in cellular biology?'}
> Finished chain.


"A question that's more biology than physics, but I'll do my best to provide an accurate and concise explanation!\n\nDNA (Deoxyribonucleic acid) plays a central role in cellular biology as the primary genetic material that contains the instructions for the development and function of all living organisms. Think of it like a blueprint or a set of instructions that are encoded on a molecule.\n\nHere's how it works: DNA is made up of four nucleotide bases - adenine (A), guanine (G), cytosine (C), and thymine (T) - which are arranged in a specific sequence to form genes. These genes contain the information needed for the cell to produce proteins, which perform various functions such as catalyzing chemical reactions, transporting molecules across cell membranes, and more.\n\nWhen a cell divides, its DNA is replicated so that each new cell receives a complete set of genetic instructions. This ensures that all cells in an organism have the same genetic makeup, allowing for the development of 